In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime

In [4]:
# For time series analysis
from tsfresh import extract_features

ModuleNotFoundError: No module named 'tsfresh'

In [ ]:
# For additional statistical analysis
import pingouin as pg
from pingouin import pairwise_ttests

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# declare a Base using 'automap_base()'
Base = automap_base()

In [ ]:
# reflect an existing database into a new model
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

# Exploratory Precipitation Analysis

In [ ]:
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
most_recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.
# Calculate the date one year from the last date in data set.

period_start = dt.date(2017,8,23) - dt.timedelta(days=365)
period_start

In [ ]:
# Perform a query to retrieve the data and precipitation scores

last_year_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= period_start).all()

last_year_data[0:5]

In [ ]:
# Save the query results as a Pandas DataFrame. Explicitly set the column names
prcp_df = pd.DataFrame(last_year_data, columns = ['Date', 'Precipitation'])
prcp_df.head()

In [ ]:
# Sort the dataframe by date and set index as 'Date'
prcp_df = prcp_df.sort_values(by=['Date']).set_index(['Date'])
prcp_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot(color = 'darkred', rot=45, figsize=[12,12])

plt.xlabel("Date")
plt.ylabel("Precipitation (Inches)")
plt.title(f"Daily Precipitation in Hawaii in Inches ({period_start} - {most_recent_date})", size=15)
plt.savefig("precipitation_plot.png")
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
prcp_df.Precipitation.describe()

In [ ]:
# Resample weekly for mean
# Note: Resampling requires input as index
df = prcp_df.copy()
df.index = pd.to_datetime(df.index)
median_df = df.resample('w').mean()
median_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
median_df.plot(color = 'darkorange', rot=90, figsize=[12,8])

plt.xlabel("Date")
plt.ylabel("Precipitation (Inches)")
plt.title(f"Mean Daily Precipitation in Hawaii Calculated by Week (Inches) ({period_start} - {most_recent_date})", size=15)
plt.savefig("mean_daily_precipitation_plot.png")
plt.show()

In [ ]:
# Use datetime and a lambda function to extract month from 'Date'
get_months = df.reset_index()
get_months["Month"] = get_months.Date.map(lambda x: x.month)
get_months.plot.box(by="Month", figsize=[12,6])

plt.xlabel("Month of Year", size=12)
plt.ylabel("Precipitation (Inches)", size=12)
plt.title("Box Plot of Hawaii Station Recordings of Daily Precipitation (Inches) by Calendar Month", size=12)
plt.savefig("monthly_precipitation_boxplot.png")
plt.show()

In [ ]:
get_months.columns

In [ ]:
# Use pingouin to perform pairwise ttests on monthly mean precipitation
pairwise = pg.pairwise_gameshowell(data=get_months, dv='Precipitation', between='Month').sort_values(['pval'], ascending = True).set_index('A')
significant_pairwise = pairwise[pairwise.pval <= .05]
significant_pairwise

In [ ]:
# Write the ttest table to a csv file
significant_pairwise.to_csv("pairwise_ttests_precipitation_by_month.csv")

### Discussion: In Hawaii, between 8/23/16 and 8/23/17, average daily precipitation in January was significantly different from that in April, September, and December. Furthermore, average daily precipitation in April was significantly different from that in June, November, and December.

### Specifically, days in January were significantly "drier" than its pairwise counterparts and days in April were significantly "wetter" than its counterparts.

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
n_stations = session.query(Station.station).count()
n_stations

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.

station_activity = session.query(Measurement.station, func.count(Measurement.id)).\
    group_by(Measurement.station).order_by(func.count(Measurement.id).desc()).all()
station_activity[0:5]

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = station_activity[0][0]
most_active_station

In [ ]:
most_active_station_temperatures = session.query(Measurement.station, Measurement.tobs).\
    filter(Measurement.station == most_active_station)
most_active_station_temperatures

In [ ]:
most_active_temp_df = pd.DataFrame(most_active_station_temperatures, columns = ['Station', 'Temperature'])
most_active_temp_df.head()

In [ ]:
most_active_temp_df.Temperature.describe()

In [ ]:
lowest_temp = most_active_temp_df.Temperature.min()
highest_temp = most_active_temp_df.Temperature.max()
average_temp = most_active_temp_df.Temperature.mean()

print(lowest_temp, highest_temp, average_temp)

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
MAS_year_query = session.query(Measurement.station, Measurement.tobs).\
    filter(Measurement.station == most_active_station).\
    filter(Measurement.date >= period_start).all()

MAS_annual_temp_df = pd.DataFrame(MAS_year_query, columns = ['Station', 'Temperature'])
MAS_annual_temp_df

In [ ]:
MAS_annual_temp_df.Temperature.plot.hist(figsize = [12,8], color = 'magenta', edgecolor = 'black')

plt.title(f"Histogram of Daily Temperature Observations (F) at Station {most_active_station} ({period_start} - {most_recent_date})", size = 15)
          
plt.xlabel("Temperature (F)")
plt.savefig("temperature_histogram.png")
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()